### 2. Exploratory Data Analysis (Data Cleaning & View Creation)
**Objective:** Transform raw OULAD data into clean, analysis-ready tables and create comprehensive views that aggregate student engagement, performance, and demographic data for analytical insights.

**Data Quality Issues Identified**

- **Date Format Problems:** Raw date columns stored as integers (days since course start) rather than proper DATE format
- **Non-numeric Values:** Some score and click columns contained non-numeric data requiring casting
- **Missing Value Handling:** Inconsistent NULL handling across related tables

**a. Data Quality Assessment**

- Identified problematic date columns in `assessments`, `student_registration`, and `student_vle` tables
- Detected non-numeric values in score and click columns requiring type conversion
- Validated data integrity before transformation

In [0]:
-- Set database context
USE eduanalytics;

-- Data Quality Check: Identify malformed data before cleaning
-- Check for non-numeric values in date columns
SELECT 'assessments' as table_name, 
       'date' as column_name,
       date as problematic_value,
       COUNT(*) as count
FROM eduanalytics.assessments 
WHERE TRY_CAST(date AS INT) IS NULL AND date IS NOT NULL
GROUP BY date

UNION ALL

SELECT 'student_registration' as table_name, 
       'date_registration' as column_name,
       date_registration as problematic_value,
       COUNT(*) as count
FROM eduanalytics.student_registration 
WHERE TRY_CAST(date_registration AS INT) IS NULL AND date_registration IS NOT NULL
GROUP BY date_registration


UNION ALL
-- Check for non-numeric values in score columns
SELECT 'student_assessment' as table_name, 
       'score' as column_name,
       score as problematic_value,
       COUNT(*) as count
FROM eduanalytics.student_assessments 
WHERE TRY_CAST(score AS DOUBLE) IS NULL AND score IS NOT NULL
GROUP BY score
LIMIT 10;

**b. Date Standardization**
- Base Date: Established 2013-02-01 as reference point for date calculations
- Conversion Logic: Used `DATE_ADD()` function to convert integer days to proper DATE format
- Tables Cleaned: `assessments_clean`, `studentregistration_clean`, `studentassessment_clean`, `studentvle_clean`

In [0]:
-- Fix assessments table
CREATE OR REPLACE TABLE eduanalytics.assessments_clean AS
SELECT 
    code_module,
    code_presentation,
    id_assessment,
    assessment_type,
    CASE 
        WHEN TRY_CAST(date AS INT) IS NOT NULL 
        THEN DATE_ADD('2013-02-01', TRY_CAST(date AS INT)) 
        ELSE NULL 
    END AS date,
    weight
FROM eduanalytics.assessments;

-- Fix student registration table  
CREATE OR REPLACE TABLE eduanalytics.studentregistration_clean AS
SELECT 
    code_module,
    code_presentation,
    id_student,
    CASE 
        WHEN TRY_CAST(date_registration AS INT) IS NOT NULL 
        THEN DATE_ADD('2013-02-01', TRY_CAST(date_registration AS INT)) 
        ELSE NULL 
    END AS date_registration,
    CASE 
        WHEN date_unregistration IS NOT NULL AND TRY_CAST(date_unregistration AS INT) IS NOT NULL
        THEN DATE_ADD('2013-02-01', TRY_CAST(date_unregistration AS INT)) 
        ELSE NULL 
    END AS date_unregistration
FROM eduanalytics.student_registration;  -- Note: using snake_case based on your workspace

-- Fix student assessment table
CREATE OR REPLACE TABLE eduanalytics.studentassessment_clean AS
SELECT 
    id_assessment,
    id_student,
    CASE 
        WHEN TRY_CAST(date_submitted AS INT) IS NOT NULL 
        THEN DATE_ADD('2013-02-01', TRY_CAST(date_submitted AS INT)) 
        ELSE NULL 
    END AS date_submitted,
    is_banked,
    TRY_CAST(score AS DOUBLE) AS score  -- Also handle score casting
FROM eduanalytics.student_assessments;  -- Note: using snake_case based on your workspace

-- Fix student VLE table (has date column)
CREATE OR REPLACE TABLE eduanalytics.studentvle_clean AS
SELECT 
    code_module,
    code_presentation,
    id_student,
    id_site,
    CASE 
        WHEN TRY_CAST(date AS INT) IS NOT NULL 
        THEN DATE_ADD('2013-02-01', TRY_CAST(date AS INT)) 
        ELSE NULL 
    END AS date,
    TRY_CAST(sum_click AS INT) AS sum_click  -- Handle sum_click casting
FROM eduanalytics.student_vle;  -- Note: using snake_case based on your workspace

-- Fix VLE table (has week_from and week_to columns - these are relative week numbers)
CREATE OR REPLACE TABLE eduanalytics.vle_clean AS  
SELECT 
    id_site,
    code_module,
    code_presentation,
    activity_type,
    week_from,
    week_to
FROM eduanalytics.vle;

**c. Comprehensive View Creation**
- `v_student_info_complete` - Enhanced student demographics with registration dates and course details
- `v_vle_engagement_summary` - VLE interaction metrics per student (clicks, resources, engagement span)
- `v_assessment_performance` - Assessment scores aggregated by type (TMA, CMA, Exam) with performance statistics
`v_student_analytics_master` - Master view combining all student data with derived metrics

In [0]:
-- Create Comprehensive Student Information View
CREATE OR REPLACE VIEW eduanalytics.v_student_info_complete AS
SELECT 
    si.code_module,
    si.code_presentation,
    si.id_student,
    si.gender,
    si.region,
    si.highest_education,
    si.imd_band,
    si.age_band,
    si.num_of_prev_attempts,
    si.studied_credits,
    si.disability,
    si.final_result,
    sr.date_registration,
    sr.date_unregistration,
    c.module_presentation_length
FROM eduanalytics.student_info si  -- Note: using snake_case
LEFT JOIN eduanalytics.studentregistration_clean sr 
    ON si.code_module = sr.code_module 
    AND si.code_presentation = sr.code_presentation 
    AND si.id_student = sr.id_student
LEFT JOIN eduanalytics.student_courses c  -- Note: check actual table name
    ON si.code_module = c.code_module 
    AND si.code_presentation = c.code_presentation;

In [0]:
-- Step 3: Create VLE Engagement Summary
CREATE OR REPLACE VIEW eduanalytics.v_vle_engagement_summary AS
SELECT 
    code_module,
    code_presentation,
    id_student,
    COUNT(DISTINCT id_site) as unique_resources_accessed,
    COUNT(*) as total_interactions,
    SUM(sum_click) as total_clicks,
    AVG(sum_click) as avg_clicks_per_interaction,
    MIN(date) as first_interaction_date,
    MAX(date) as last_interaction_date,
    DATEDIFF(MAX(date), MIN(date)) as engagement_span_days
FROM eduanalytics.studentvle_clean
WHERE sum_click > 0
GROUP BY code_module, code_presentation, id_student;

-- Step 4: Create Assessment Performance Summary
CREATE OR REPLACE VIEW eduanalytics.v_assessment_performance AS
SELECT 
    sa.id_student,
    COUNT(*) as total_assessments_submitted,
    COUNT(CASE WHEN sa.score >= 40 THEN 1 END) as assessments_passed,
    AVG(sa.score) as average_score,
    MAX(sa.score) as highest_score,
    MIN(sa.score) as lowest_score,
    STDDEV(sa.score) as score_std_dev,
    COUNT(CASE WHEN a.assessment_type = 'TMA' THEN 1 END) as tma_count,
    COUNT(CASE WHEN a.assessment_type = 'CMA' THEN 1 END) as cma_count,
    COUNT(CASE WHEN a.assessment_type = 'Exam' THEN 1 END) as exam_count,
    AVG(CASE WHEN a.assessment_type = 'TMA' THEN sa.score END) as avg_tma_score,
    AVG(CASE WHEN a.assessment_type = 'CMA' THEN sa.score END) as avg_cma_score,
    AVG(CASE WHEN a.assessment_type = 'Exam' THEN sa.score END) as avg_exam_score
FROM eduanalytics.studentassessment_clean sa
JOIN eduanalytics.assessments_clean a ON sa.id_assessment = a.id_assessment
WHERE sa.score IS NOT NULL
GROUP BY sa.id_student;

In [0]:
-- Step 5: Create Master Analytics View
CREATE OR REPLACE VIEW eduanalytics.v_student_analytics_master AS
SELECT 
    si.code_module,
    si.code_presentation,
    si.id_student,
    si.gender,
    si.region,
    si.highest_education,
    si.imd_band,
    si.age_band,
    si.num_of_prev_attempts,
    si.studied_credits,
    si.disability,
    si.final_result,
    si.date_registration,
    si.date_unregistration,
    si.module_presentation_length,
    
    -- VLE Engagement Metrics
    COALESCE(ve.unique_resources_accessed, 0) as unique_resources_accessed,
    COALESCE(ve.total_interactions, 0) as total_vle_interactions,
    COALESCE(ve.total_clicks, 0) as total_clicks,
    COALESCE(ve.avg_clicks_per_interaction, 0) as avg_clicks_per_interaction,
    ve.first_interaction_date,
    ve.last_interaction_date,
    COALESCE(ve.engagement_span_days, 0) as engagement_span_days,
    
    -- Assessment Performance Metrics
    COALESCE(ap.total_assessments_submitted, 0) as total_assessments_submitted,
    COALESCE(ap.assessments_passed, 0) as assessments_passed,
    ap.average_score,
    ap.highest_score,
    ap.lowest_score,
    ap.score_std_dev,
    COALESCE(ap.tma_count, 0) as tma_count,
    COALESCE(ap.cma_count, 0) as cma_count,
    COALESCE(ap.exam_count, 0) as exam_count,
    ap.avg_tma_score,
    ap.avg_cma_score,
    ap.avg_exam_score,
    
    -- Derived Metrics
    CASE 
        WHEN ap.total_assessments_submitted > 0 
        THEN (ap.assessments_passed * 100.0 / ap.total_assessments_submitted)
        ELSE 0 
    END as pass_rate_percentage,
    
    CASE 
        WHEN ve.total_clicks > 1000 THEN 'High'
        WHEN ve.total_clicks > 500 THEN 'Medium'
        WHEN ve.total_clicks > 0 THEN 'Low'
        ELSE 'No Engagement'
    END as engagement_level

FROM eduanalytics.v_student_info_complete si
LEFT JOIN eduanalytics.v_vle_engagement_summary ve 
    ON si.code_module = ve.code_module 
    AND si.code_presentation = ve.code_presentation 
    AND si.id_student = ve.id_student
LEFT JOIN eduanalytics.v_assessment_performance ap 
    ON si.id_student = ap.id_student;

In [0]:
-- Step 6: Data Quality Check Queries
-- Run these to verify your data cleaning worked correctly

-- Check date formatting
SELECT 'Assessment Dates' as table_name, 
       COUNT(*) as total_records,
       COUNT(date) as valid_dates,
       COUNT(*) - COUNT(date) as null_dates
FROM eduanalytics.assessments_clean
UNION ALL
SELECT 'Registration Dates' as table_name,
       COUNT(*) as total_records,
       COUNT(date_registration) as valid_dates,
       COUNT(*) - COUNT(date_registration) as null_dates
FROM eduanalytics.studentregistration_clean
UNION ALL
SELECT 'VLE Dates' as table_name,
       COUNT(*) as total_records,
       COUNT(date) as valid_dates,
       COUNT(*) - COUNT(date) as null_dates
FROM eduanalytics.studentvle_clean;

In [0]:
-- Check master view record counts
SELECT 
    'Total Students' as metric,
    COUNT(*) as count
FROM eduanalytics.v_student_analytics_master
UNION ALL
SELECT 
    'Students with VLE Data' as metric,
    COUNT(*) as count
FROM eduanalytics.v_student_analytics_master 
WHERE total_clicks > 0
UNION ALL
SELECT 
    'Students with Assessment Data' as metric,
    COUNT(*) as count
FROM eduanalytics.v_student_analytics_master 
WHERE total_assessments_submitted > 0;


In [0]:
-- Sample data verification
SELECT 
    code_module,
    code_presentation,
    final_result,
    COUNT(*) AS student_count,
    ROUND(AVG(total_clicks), 2) AS avg_clicks,
    ROUND(AVG(average_score), 2) AS avg_score
FROM eduanalytics.v_student_analytics_master
GROUP BY code_module, code_presentation, final_result
ORDER BY code_module, code_presentation, final_result
LIMIT 20;